In [1]:
import requests
from bs4 import BeautifulSoup as bs
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.exceptions import IllegalCharacterError
import urllib.request
import urllib.error
import time

to_scrape = "https://www.taptap.com/topic/3805806"

In [2]:
class TapTapScraper:
    def __init__(self, url):
        self.wb = Workbook()
        self.ws = self.wb.active
        self.ws.title = "TapTap"
        self.next_row = 1
        self.row_dict = {1: 1}
        self.url = url

    def parse_page(self, page_num):
        page = self.get_page(page_num)

        # get pages
        if page_num in self.row_dict.keys():
            self.next_row = self.row_dict[page_num]
        while page.status_code != 200:
            time.sleep(1)
            page = self.get_page(page_num)
        page_soup = bs(page.text, 'html.parser')

        commentList = page_soup.find_all(class_="posts-item-text topic-posts-item-text")
        for comment in commentList:
            # parse comment and secondary comments
            parsed_comments = self.parse_primary_comment(comment)
            for parsed_comment in parsed_comments:
                for i in range(len(parsed_comment)):
                    try:
                        self.ws.cell(row=self.next_row, column=(i + 1), value=parsed_comment[i])
                    except IllegalCharacterError:
                        pass
                self.next_row += 1
        self.row_dict[page_num + 1] = self.next_row
        time.sleep(1)

    def parse_primary_comment(self, comment):
        """
        parses primary and secondary comments
        :param comment: 
        :return list of parsed primary and secondary comments: 
        """

        comment_soup = bs(str(comment), 'html.parser')
        parsed_comments = []

        # primary comment
        tap_user_id = str(comment_soup.find_all(class_='taptap-user')[0]).split('"')[3]
        tap_user_name = comment_soup.find_all(class_="taptap-user-name taptap-link")[0].get_text().strip()
        primary_floor_num = int(comment_soup.find_all(class_="pull-right")[0].get_text()[:-1])
        comment_body = comment_soup.find_all(class_="item-text-body bbcode-body js-open-bbcode-image")[0]
        comment_text = comment_body.prettify()
        comment_text = comment_text.replace("<br/>", "")
        comment_text = remove_tags(comment_text).strip()
        parsed_comments.append((tap_user_id, tap_user_name, primary_floor_num, comment_text))

        #secondary comments
        secondary_comment_list = comment.find_all(class_="comment-item-text")
        collapsed_secondary_comment_links = comment.find_all(class_="pagination")
        if not collapsed_secondary_comment_links:
            return parsed_comments
        collapsed_secondary_comment_links = comment.find_all(class_="pagination")[0].find_all("a")  # list of <a> tags
        collapsed_secondary_comment_links = list(map(lambda x: x["href"], collapsed_secondary_comment_links))[
                                            :-1]  # list of links, for some reason there is an extra copy of page 2 at the end
        for link in collapsed_secondary_comment_links:
            response = requests.get(link)
            response_text = response.json()["data"]["html"]
            response_soup = bs(response_text, "html.parser")
            response_comment_list = response_soup.find_all(class_="taptap-comment-item")
            for response_comment in response_comment_list:
                secondary_comment_list.append(response_comment)
        for i, secondary_comment in enumerate(secondary_comment_list):
            parsed_comments.append(
                Scraper.parse_secondary_comment(secondary_comment, str(primary_floor_num) + "_" + str(i + 1)))

        return parsed_comments

    def parse_secondary_comment(self, secondary_comment, floor_num):
        tap_user = secondary_comment.find_all(class_="taptap-user")[0]
        tap_user_id = tap_user["data-user-id"]
        tap_user_name = tap_user.text.strip()
        floor_num = floor_num
        comment_text = secondary_comment.find_all(class_="item-text-body")[0].text.strip()

        return tap_user_id, tap_user_name, floor_num, comment_text


    def save(self, checkpoint=None):
        if not checkpoint:
            self.wb.save('TapTap_Weapon_Description.xlsx')
        else:
            self.wb.save('TapTap_Weapon_Description_%d.xlsx' % checkpoint)

    def get_page(self, page_num):
        url = self.url + "?page=%d#postsList" % page_num
        page = requests.get(url)
        return page

def remove_tags(text):
    while "<" in text:
        start = text.find("<")
        end = text.find(">")
        text = text[:start] + text[end + 1:]
    return text


In [3]:
Scraper = TapTapScraper(to_scrape)

In [4]:
i = 1
while i < 196:
    try:
        Scraper.parse_page(i)
        print("Scraped page %d" % i)
        Scraper.save()
        i += 1
    except urllib.error.HTTPError as e:
        print("HTTPError detected, re-scraping page")
        continue
    except ConnectionError as e:
        time.sleep(20)
        continue
    if i % 30 == 0:
        Scraper.save(i)


Scraped page 1


Scraped page 2


Scraped page 3


Scraped page 4


Scraped page 5


...

Scraped page 191


Scraped page 192


Scraped page 193


Scraped page 194


Scraped page 195


In [88]:
page = Scraper.get_page(1)
page_soup = bs(page.text, "html.parser")
# 一级回复
commentList = page_soup.find_all(class_="posts-item-text topic-posts-item-text")
comment = commentList[16]


In [57]:
# 二级回复
secondary_comment_list = comment.find_all(class_="comment-item-text")
collapsed_secondary_comment_links = comment.find_all(class_="pagination")[0].find_all("a")  # list of <a> tags
collapsed_secondary_comment_links = list(map(lambda x: x["href"], collapsed_secondary_comment_links))[
                                    :-1]  # list of links, for some reason there is an extra copy of page 2 at the end
for link in collapsed_secondary_comment_links:
    response = requests.get(link)
    response_text = response.json()["data"]["html"]
    response_soup = bs(response_text,"html.parser")
    response_comment_list = response_soup.find_all(class_="taptap-comment-item")
    for response_comment in response_comment_list:
        secondary_comment_list.append(response_comment)
parsed_secondary_comments = []
for i, secondary_comment in enumerate(secondary_comment_list):
    parsed_secondary_comments.append(Scraper.parse_secondary_comment(secondary_comment, primary_floor_num + str(i+1)))
parsed_secondary_comments

[('16078861', '手机用户16078861', 0, 'em没有第二拳'),
 ('27858300', '真-无聊', 1, '2.一拳:      第一滴血'),
 ('2895597', 'AriTaliao', 2, 'AriTaliao2.一拳    那么..你变秃了嘛?'),
 ('19246', '迪拉达( ¨̮ )', 3, '没有什么是一拳解决不了的，如果有，我也没办法┐（─__─）┌'),
 ('30392201', '黄家欣', 4, '一拳秒杀？  吊炸天BOSS你秒给我看'),
 ('26634634', '雷电萨满杨永信', 5, '一拳：玩胖次！'),
 ('30392814', 'epic sans', 6, '一个光头！'),
 ('4831460', 'Be Alright', 7, '拿小拳拳锤你胸口'),
 ('27177804', 'pencake', 8, '借楼'),
 ('30396492', '手机用户30396492', 9, '我这一拳下来你可能会死'),
 ('18434247', '一点星源吧', 10, '一拳：我变秃了也变强了'),
 ('9791115', 'QQ用户9791115', 11, '只有光头才能使用的武器（手动滑稽）'),
 ('30408645', '小房子', 12, '一拳：闪瞎你的24k纯金狗眼'),
 ('2293495', '我', 13, '一拳：我这一拳下去你可能会死'),
 ('30418258', 'Jesses```Steve`', 14, '浮游炮:我很飘也很吊'),
 ('24297030', '梦逖安', 15, '打什么都一拳'),
 ('1603253', '武禁', 16, '秃头披风侠使用的武器'),
 ('9047436', 'toby papyrus', 17, '一拳   别摘我假发，我怕闪到你'),
 ('30668042', '㏒', 18, '无需多言，只需一拳。。'),
 ('25881571', '🎖🎲Jerry🎲🎖', 19, 'Jerry2.一拳 小拳拳锤爆你胸口')]

In [42]:
# 折叠二级回复
collapsed_secondary_comment_links = comment.find_all(class_="pagination")[0].find_all("a")  # list of <a> tags
collapsed_secondary_comment_links = list(map(lambda x: x["href"], collapsed_secondary_comment_links))[
                                    :-1]  # list of links, for some reason there is an extra copy of page 2 at the end
collapsed_secondary_comment_links


['https://www.taptap.com/ajax/post/comments?id=29805385&page=2',
 'https://www.taptap.com/ajax/post/comments?id=29805385&page=3',
 'https://www.taptap.com/ajax/post/comments?id=29805385&page=4']

('18434247', '一点星源吧', None, '一拳：我变秃了也变强了')